In [12]:
import numpy as np
import hapke
import matplotlib
import matplotlib.pyplot as plt
from scipy import optimize
from matplotlib.collections import PatchCollection
from mpl_toolkits import mplot3d
from scipy.interpolate import interp1d
from pyvims import VIMS
from pyvims.misc import MAPS
import random
from pyvims.misc import Map
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from pathlib import Path
from pyvims.wget import wget
matplotlib.use('Qt5Agg')
from scipy import stats
plt.rcParams.update({'font.size': 16})

colors = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

def first_degree(w, a, b):
    return a + b * w

def second_degree(w, a1, b1, c1):
    return a1 + b1 * w + c1 * w ** 2


Load cube (using hapke.retrieve_cube) and optical constants (hapke.opticalconstants)

In [2]:
T = 120
N = 20

opt_c = hapke.opticalconstants(T,sensitivity = N)
n_c = opt_c['n']
k_c = opt_c['k']
wav_c = opt_c['wav']

opt_a = hapke.opticalconstants(T, sensitivity= N,crystallinity=False)
n_a = opt_a['n']
k_a = opt_a['k']
wav_a = opt_a['wav']

int_opt = hapke.inter_optical_constants(wav_c, wav_a, n_c, k_c)

wav = np.array(int_opt['wav'])
n_c = int_opt['n']
k_c = int_opt['k']

opt_constants = [n_c,k_c,n_a,k_a]

rhea = {}

rhea_keys = {}


Read ID of all the different cubes in a txt located in the data folder

In [3]:
flyby = ['R1','R4']

for t in flyby:
    print(t)
    file_path = 'C:/Users/USUARIO/Desktop/MSc Thesis/Phase A - Data Analysis/Data/Rhea/' + t + '.txt'
    root = 'C:/Users/USUARIO/Desktop/MSc Thesis/Phase A - Data Analysis/Data/Rhea/' + t

    rhea[t] = hapke.read_cube(file_path,root)
    rhea_keys[t] = [key for key in rhea[t]]

R1
R4


In [4]:
# STUDY ALONG THE LON LINE

lat = 0
#lon_range = [-160,-140,-120,-100,-80,-60,-40,-20,0,20,40,60,80,100,120,140,160,180]

lon_range = [340,320,300,280,260,240,220,200,180,160,140,120,100,80,60,40,20,0]

is_pixel = {}
pixel = {}
pixel_key = {}
pix_flyby = {}



radius = rhea[flyby[0]][rhea_keys[flyby[0]][0]].target_radius

for lon in lon_range:

    aux_is = []
    aux_pixel = []
    aux_key = []
    aux_flyby = []

    for f in flyby:
        print(f)
        for key in rhea_keys[f]:

            pix = rhea[f][key].get_pixel(lon,lat)

            if pix is not None and hapke.is_micron_dip(pix) and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.area < (radius/7) ** 2 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:


                aux_pixel.append(pix)
                aux_key.append(key)
                aux_flyby.append(f)

    pixel[lon] = aux_pixel
    pixel_key[lon] = aux_key
    pix_flyby[lon] = aux_flyby


    print(pix_flyby[lon])

R1
R4
['R4', 'R4', 'R4']
R1
R4
['R4']
R1
R4
[]
R1
R4
[]
R1
R4
['R1']
R1
R4
['R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1']
R1
R4
['R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1']
R1
R4
['R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1']
R1
R4
['R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1']
R1
R4
['R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1']
R1
R4
['R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1']
R1
R4
['R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1']
R1
R4
['R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R1']
R1
R4
['R1', 'R1', 'R1', 'R1', 'R1', 'R1', 'R4'

In [6]:
# SPECTRUM PLOT FOR LON STUDY

fig, ax = plt.subplots()
ax.set_xlabel(r'Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_title('')
ax.grid()

for k in lon_range:
    for p in range(len(pixel[k])):

        ax.plot(pixel[k][p].wvlns,pixel[k][p].spectrum)

plt.show()


KeyboardInterrupt



In [13]:
# STUDY ALONG THE LON LINE

lat_range  = [-60,-30,0,30,60]
lon_range = [-160,-140,-120,-100,-80,-60,-40,-20,0,20,40,60,80,100,120,140,160,180]



bg = MAPS['RHEA']
fig, ax = bg.figure(figsize=(20, 10))


radius = rhea[flyby[0]][rhea_keys[flyby[0]][0]].target_radius

for lat in lat_range:
    pixel = {}
    for lon in lon_range:

        aux_pixel = []

        for f in flyby:
            print(f)
            for key in rhea_keys[f]:

                pix = rhea[f][key].get_pixel(lon,lat)

                if pix is not None and hapke.is_micron_dip(pix) and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.area < (radius/7) ** 2 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:


                    aux_pixel.append(pix)

        pixel[lon] = aux_pixel

    patches = []

    for k in lon_range:
        for p in range(len(pixel[k])):
            hapke.generate_patches(patches,pixel[k][p])

    pc = PatchCollection(patches, alpha=0.4, edgecolor='yellow', linewidth=2)

    ax.add_collection(pc)
    print(lat)
plt.title('Rhea')
plt.show()

R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
-60
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
-30
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
0
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
30
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
R1
R4
60



KeyboardInterrupt



In [11]:
# BG PLOT FOR LON STUDY

bg = MAPS[rhea[flyby[0]][rhea_keys[flyby[0]][0]].target_name]
fig, ax = bg.figure(figsize=(20, 10))

patches = []


for k in lon_range:
    for p in range(len(pixel[k])):
        hapke.generate_patches(patches,pixel[k][p])

pc = PatchCollection(patches, alpha=0.4, edgecolor='yellow', linewidth=2)

ax.add_collection(pc)
plt.title('Rhea Pixels')
plt.show()


KeyboardInterrupt



In [36]:
# SPECTRUM PLOT FOR LON STUDY

fig, ax = plt.subplots()
ax.set_xlabel(r'Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_title('')
ax.grid()

for k in lon_range:
    for p in range(len(pixel[k])):
        print(pixel[k][p].area)
        print(pixel[k][p].inc)
        print('')
        if p == 16:

            print('16!!')
            ax.plot(pixel[k][p].wvlns,pixel[k][p].spectrum, c = colors[1],ls='--',label='Pixel 17')
        if p == 17:

            ax.plot(pixel[k][p].wvlns,pixel[k][p].spectrum, c = colors[2],ls=':',label='Pixel 18')
        else:
            ax.plot(pixel[k][p].wvlns,pixel[k][p].spectrum, c = colors[0],linewidth=1)


plt.legend()
plt.show()

11719.026690016868
33.112214741080585

11243.380420532942
32.489378647278656

11154.734160442054
32.43018033538986

4100.621378290965
28.98522523609838

3732.101133968412
27.784283774765484

3559.6880639241435
27.027854509227772

3240.9553038027034
26.948876629897644

3080.2426928611435
26.161004857065908

2793.557017271491
25.911643519302096

2732.845214580811
25.363682738187833

2648.266567194876
25.77716303916964

2403.8401356465397
25.17397686869487

2328.6350272013797
24.913580236307805

1947.62486642901
24.081172040814156

1828.929156646514
23.862506358481536

2355.535364553448
22.91044682819595

1729.9384348058004
20.523896690721084

16!!
1683.7088923351191
21.917106752066477

570.3413455590703
20.803524834900077

523.8768539829586
20.04034019331701

358.5940432473566
19.573350692869898




KeyboardInterrupt



In [5]:
# FIT FOR LON STUDY
n_fits = 1

D_fit = {}
error_D = {}
theta_fit = {}
error_theta = {}
legend_fit = {}
legend_cryst = {}

m_am = {}
err_m_am = {}

cryst_area_norm_1 = {}
cryst_2_loc = {}
err_cryst_2_loc = {}
area = {}
aux_a = 0
auxxxx = 0

for k in lon_range:
    print(k)

    aux_D_fit = []
    aux_error_D = []
    aux_theta_fit = []
    aux_error_theta = []
    aux_legend_fit = []
    aux_legend_cryst = []

    aux_cryst_area_norm_1 = []
    aux_cryst_2_loc = []
    aux_err_cryst_2_loc = []
    aux_area = []

    aux_m = []
    aux_err_m = []

    for p in range(len(pixel[k])):
        auxxxx +=1
        aux_area.append(pixel[k][p].area)

        IF = pixel[k][p].spectrum
        wav1 = pixel[k][p].wvlns

        angles = [np.deg2rad(pixel[k][p].eme),np.deg2rad(pixel[k][p].inc),np.deg2rad(pixel[k][p].phase)]

        random_fit = hapke.random_fit(n_fits,wav, angles, IF, wav1,*opt_constants)

        if not random_fit['Unique']:
            aux_a += 1

        #print([random_fit['dif'],random_fit['n_sol'],random_fit['Unique']])

        fit = random_fit['fit']

        model_IF = hapke.hapke_model_mixed(fit.x,wav,angles,*opt_constants)['IF']

        # CALCULATE ERRORS
        errors = hapke.print_error_correlation(fit)

        #if errors[0] < fits[pixel_key[p]].x[0] / 2 and errors[1] < fits[pixel_key[p]].x[1] / 2:
        #    a = 1
        aux_error_D.append(errors[1])
        aux_error_theta.append(errors[0])


        aux_legend_fit.append(pixel_key[k][p])
        aux_D_fit.append(fit.x[1])
        aux_theta_fit.append(fit.x[0])

        aux_legend_cryst.append(pixel_key[k][p])

        aux_cryst_area_norm_1.append(hapke.cryst_area_one_peak(IF,wav1)['area'] / hapke.cryst_area_one_peak(model_IF,wav)['area'])

        loc_2 = hapke.cryst_area(IF,wav1)['loc_2']

        aux_cryst_2_loc.append(loc_2[0])
        aux_err_cryst_2_loc.append(loc_2[1])

        aux_m.append(hapke.loc_to_m(loc_2[0],loc_2[1])[0])
        aux_err_m.append(hapke.loc_to_m(loc_2[0],loc_2[1])[1])

    D_fit[k] = aux_D_fit
    error_D[k] = aux_error_D
    theta_fit[k] = aux_theta_fit
    error_theta[k] = aux_error_theta
    legend_fit[k] = aux_legend_fit
    legend_cryst[k] = aux_legend_cryst

    cryst_area_norm_1[k] = aux_cryst_area_norm_1
    cryst_2_loc[k] = aux_cryst_2_loc
    err_cryst_2_loc[k] = aux_err_cryst_2_loc
    area[k] = aux_area

    m_am[k] = aux_m
    err_m_am[k] = aux_err_m


340
[1.802459417979299]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.82927146]
 [-0.82927146  1.        ]]


C:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


[1.8656632752665736]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.82928699]
 [-0.82928699  1.        ]]
[1.9462002835329932]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.82830566]
 [-0.82830566  1.        ]]
320
[1.8990411942779992]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.83020616]
 [-0.83020616  1.        ]]
300
280
260
[1.897866020108113]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.82426045]
 [-0.82426045  1.        ]]
240
[2.0683623755762253]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.79676906]
 [-0.79676906  1.        ]]
[3.157988445608748]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.97399282]
 [-0.97399282  1.        ]]
[2.2306405697788767]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.97225168]
 [-0.97225168  1.        ]]
[2.0020137303418233]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.97012944]
 [-0.97012944  1.        ]]
[2.4208304577439]
[0.0, 0.0, 0.0]
Correlation matrix:
[[ 1.         -0.97183472]
 [-0.

In [58]:
#PLOT VALUES OF ONE LOCATION

fig, (ax1,ax3) = plt.subplots(2,1, sharex=True,figsize=(6,4))

k = 90

ax1.errorbar(range(len(D_fit[k])), np.array(D_fit[k])*1000000,yerr=np.array(error_D[k])*1000000, color=colors[0], fmt='-o')


ax1.set_ylabel(r'$D$ [$\mu$m]', color=colors[0])
ax1.tick_params(axis='y', labelcolor=colors[0])

ax2 = ax1.twinx()

ax2.errorbar(range(len(D_fit[k])), np.rad2deg(np.array(theta_fit[k])),yerr=np.rad2deg(np.array(error_theta[k])), color=colors[1], fmt='-x')

ax2.set_ylabel(r'$\bar{\theta}$ [deg]', color=colors[1])
ax2.tick_params(axis='y', labelcolor=colors[1])
ax1.grid()


ax3.errorbar(range(len(D_fit[k])), cryst_area_norm_1[k], color=colors[0], fmt='-o')

ax3.set_xlabel('Longitude [deg]')
ax3.set_ylabel(r'Area [-]', color=colors[0])
ax3.tick_params(axis='y', labelcolor=colors[0])

ax4 = ax3.twinx()

ax4.errorbar(range(len(D_fit[k])), m_am[k],yerr=err_m_am[k], color=colors[1], fmt='-x')

ax4.set_ylabel(r'$m$ [-]', color=colors[1])
ax4.tick_params(axis='y', labelcolor=colors[1])
ax3.grid()


ax1.set_title(r'')


plt.show()


KeyboardInterrupt



In [47]:

fig, (ax1,ax3) = plt.subplots(2,1, sharex=True,figsize=(6,4))

k=180
x_ax = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21']
ax1.errorbar(x_ax, np.array(D_fit[k])*1000000,yerr=np.array(error_D[k])*1000000, color=colors[0], fmt='-o')

#ax1.set_xlabel('Longitude [deg]')
ax1.set_ylabel(r'$D$ [$\mu$m]', color=colors[0])
ax1.tick_params(axis='y', labelcolor=colors[0])

#area_1_peak_norm[13] = np.nan
#err_area_1_peak_norm[13] = np.nan

ax2 = ax1.twinx()

ax2.errorbar(x_ax, np.rad2deg(np.array(theta_fit[k])),yerr=np.rad2deg(np.array(error_theta[k])), label='Data 2', color=colors[1], fmt='-x')

ax2.set_ylabel(r'$\bar{\theta}$ [deg]', color=colors[1])
ax2.tick_params(axis='y', labelcolor=colors[1])
ax1.grid()

#area_1_peak_norm[6] = np.nan
#err_area_1_peak_norm[6] = np.nan

ax3.errorbar(x_ax, cryst_area_norm_1[k],color=colors[0], fmt='-o')

ax3.set_xlabel('Pixel Number')
ax3.set_ylabel(r'Area [-]', color=colors[0])
ax3.tick_params(axis='y', labelcolor=colors[0])

ax4 = ax3.twinx()

ax4.errorbar(x_ax, m_am[k], yerr = err_m_am[k], color=colors[1], fmt='-x')

ax4.set_ylabel(r'$m$ [-]', color=colors[1])
ax4.tick_params(axis='y', labelcolor=colors[1])
ax3.grid()

ax1.set_title(r'')


plt.show()

KeyboardInterrupt: 

In [6]:
# CALCULATE AVERAGE FOR EACH VALUE

ar = []
D = []
theta = []
loc_2m = []
area_1_peak_norm = []

n_pixels = []

err_ar = []
err_D = []
err_theta = []
err_loc_2m = []
err_area_1_peak_norm = []

m = []
err_m = []

for k in lon_range:

    n_pixels.append(len(legend_cryst[k]))

    aux = hapke.array_mean(area[k])
    ar.append(aux[0])
    err_ar.append(aux[1])

    aux = hapke.array_mean(D_fit[k], error_D[k])
    D.append(aux[0])
    err_D.append(aux[1])

    aux = hapke.array_mean(theta_fit[k], error_theta[k])
    theta.append(aux[0])
    err_theta.append(aux[1])

    aux = hapke.array_mean(cryst_2_loc[k],err_cryst_2_loc[k])
    loc_2m.append(aux[0])
    err_loc_2m.append(aux[1])

    m.append(hapke.loc_to_m(aux[0],aux[1])[0])
    err_m.append(hapke.loc_to_m(aux[0],aux[1])[1])

    aux = hapke.array_mean(cryst_area_norm_1[k])
    area_1_peak_norm.append(aux[0])
    err_area_1_peak_norm.append(aux[1])

C:\Users\USUARIO\PycharmProjects\pythonProject\hapke.py:751: RuntimeWarning: invalid value encountered in scalar divide
  sample_std_dev = np.sqrt(np.sum((arr - mean) ** 2) / (n - 1))
C:\Users\USUARIO\PycharmProjects\pythonProject\hapke.py:750: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(arr) / n
C:\Users\USUARIO\PycharmProjects\pythonProject\hapke.py:752: RuntimeWarning: invalid value encountered in scalar divide
  mean_error = sample_std_dev / np.sqrt(n)
C:\Users\USUARIO\PycharmProjects\pythonProject\hapke.py:755: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(arr / error ** 2) / np.sum(1 / error ** 2)
C:\Users\USUARIO\PycharmProjects\pythonProject\hapke.py:756: RuntimeWarning: divide by zero encountered in scalar divide
  mean_error = 1 / np.sqrt(np.sum(1 / error ** 2))


In [ ]:
#PLOT MEAN VALUES ALONG LON

fig, (ax1,ax3) = plt.subplots(2,1, sharex=True,figsize=(6,4))

ax1.errorbar(lon_range, np.array(D)*1000000,yerr=np.array(err_D)*1000000, label='Data 1', color=colors[0], fmt='-o')

#ax1.set_xlabel('Longitude [deg]')
ax1.set_ylabel(r'$D$ [$\mu$m]', color=colors[0])
ax1.tick_params(axis='y', labelcolor=colors[0])

#area_1_peak_norm[13] = np.nan
#err_area_1_peak_norm[13] = np.nan

ax2 = ax1.twinx()

ax2.errorbar(lon_range, np.rad2deg(np.array(theta)),yerr=np.rad2deg(np.array(err_theta)), label='Data 2', color=colors[1], fmt='-x')

ax2.set_ylabel(r'$\bar{\theta}$ [deg]', color=colors[1])
ax2.tick_params(axis='y', labelcolor=colors[1])
ax1.grid()

#area_1_peak_norm[6] = np.nan
#err_area_1_peak_norm[6] = np.nan

ax3.errorbar(lon_range, area_1_peak_norm,yerr=err_area_1_peak_norm, color=colors[0], fmt='-o')

ax3.set_xlabel('Longitude [deg]')
ax3.set_ylabel(r'Area [-]', color=colors[0])
ax3.tick_params(axis='y', labelcolor=colors[0])

ax4 = ax3.twinx()

ax4.errorbar(lon_range, m, yerr = err_m, color=colors[1], fmt='-x')

ax4.set_ylabel(r'$m$ [-]', color=colors[1])
ax4.tick_params(axis='y', labelcolor=colors[1])
ax3.grid()

ax1.set_title(r'Rhea 30$^\circ$S')


plt.show()

In [79]:
print(lat)

-70


In [93]:
var = np.array(D)*1000000
p_var = D_fit

err = np.array(err_D)*1000000

for k in reversed(range(len(lon_range))):

    if len(p_var[lon_range[k]]) > 3:
        a , p = stats.shapiro(np.array(p_var[lon_range[k]]))
        if p < 0.05:
            print(f'{var[k]:.0f}' + f' ± {err[k]:.0f}' + str(' F'))

        else:
            print(f'{var[k]:.0f}' + f' ± {err[k]:.0f}')

    else:
        print(f'{var[k]:.0f}' + f' ± {err[k]:.0f}')

61 ± 8 F


In [94]:
var = np.rad2deg(np.array(theta))
p_var = theta_fit

err = np.rad2deg(np.array(err_theta))

for k in reversed(range(len(lon_range))):

    if len(p_var[lon_range[k]]) > 3:
        a , p = stats.shapiro(np.array(p_var[lon_range[k]]))
        if p < 0.05:
            print(f'{var[k]:.1f}' + f' ± {err[k]:.1f}' + str(' F'))

        else:
            print(f'{var[k]:.1f}' + f' ± {err[k]:.1f}')

    else:
        print(f'{var[k]:.1f}' + f' ± {err[k]:.1f}')

60.2 ± 3.0


In [95]:
var = loc_2m
p_var = cryst_2_loc

err = err_loc_2m

for k in reversed(range(len(lon_range))):

    if len(p_var[lon_range[k]]) > 3:
        a , p = stats.shapiro(np.array(p_var[lon_range[k]]))
        if p < 0.05:
            print(f'{var[k]:.4f}' + f' ± {err[k]:.4f}' + str(' F'))

        else:
            print(f'{var[k]:.4f}' + f' ± {err[k]:.4f}')

    else:
        print(f'{var[k]:.4f}' + f' ± {err[k]:.4f}')

2.0259 ± 0.0004


In [96]:
var = area_1_peak_norm
p_var = cryst_area_norm_1

err = err_area_1_peak_norm

for k in reversed(range(len(lon_range))):

    if len(p_var[lon_range[k]]) > 3:
        a , p = stats.shapiro(np.array(p_var[lon_range[k]]))
        if p < 0.05:
            print(f'{var[k]:.2f}' + f' ± {err[k]:.2f}' + str(' F'))

        else:
            print(f'{var[k]:.2f}' + f' ± {err[k]:.2f}')

    else:
        print(f'{var[k]:.2f}' + f' ± {err[k]:.2f}')

0.86 ± 0.04


In [97]:
var = m
p_var = cryst_2_loc

err = err_m

for k in reversed(range(len(lon_range))):

    if len(p_var[lon_range[k]]) > 3:
        a , p = stats.shapiro(np.array(p_var[lon_range[k]]))
        if p < 0.05:
            print(f'{var[k]:.2f}' + f' ± {err[k]:.2f}' + str(' F'))

        else:
            print(f'{var[k]:.2f}' + f' ± {err[k]:.2f}')

    else:
        print(f'{var[k]:.2f}' + f' ± {err[k]:.2f}')

0.20 ± 0.02


Select location and record all pixels in the area fulfilling the selection criteria:
- No 1-micron dip
- Pixel not at the limb of the cube
- Intensity greater than 0.05
- Area smaller than the square than a fifth of the radius
- Incidence and emergence smaller than 90º
- Phase greater than 5º

In [36]:
lat = 90
lon = 0

is_pixel = []
pixel = []

pixel_key = []

pixel_flyby = []

radius = rhea['R1'][rhea_keys['R1'][0]].target_radius

for f in flyby:
    print(f)
    for key in rhea_keys[f]:

        pix = rhea[f][key].get_pixel(lon,lat)

        if pix is not None and hapke.is_micron_dip(pix) and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.area < (radius/7) ** 2 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:
            is_pixel.append(True)
            pixel.append(pix)
            pixel_key.append(key)
            pixel_flyby.append(f)

    print(pixel_flyby)

R1
[]
R4
['R4', 'R4', 'R4', 'R4', 'R4', 'R4', 'R4', 'R4']


In [10]:
print(pixel_key)

['1511714234_1', '1511715058_1', '1511715575_5', '1511716399_1', '1511716925_1', '1511717749_1', '1511718010_1', '1511718311_1', '1511719135_1', '1511719382_1', '1511720630_1', '1511721288_1', '1511724178_1', '1511726125_1', '1511726273_1']


Generate map of pixels and pixel coverage

In [ ]:
bg = MAPS['RHEA']
fig, ax = bg.figure(figsize=(20, 10))

lonlat = [[0,0],[90,0],[180,0],[-100,0],[0,-70],[180,-70],[0,70],[180,70]]

for t in lonlat:

    is_pixel = []
    pixel = []

    pixel_key = []

    pixel_flyby = []

    radius = rhea['R1'][rhea_keys['R1'][0]].target_radius

    for f in flyby:
        print(f)
        for key in rhea_keys[f]:

            pix = rhea[f][key].get_pixel(*t)

            if pix is not None and hapke.is_micron_dip(pix) and not pix.limb and np.mean(np.array(pix.spectrum)) > 0.025 and pix.area < (radius/7) ** 2 and pix.inc < 90 and pix.phase > 5 and pix.eme < 90:
                is_pixel.append(True)
                pixel.append(pix)
                pixel_key.append(key)
                pixel_flyby.append(f)

    patches = []

    for p in range(len(pixel)):
        hapke.generate_patches(patches,pixel[p])

    pc = PatchCollection(patches, alpha=0.4, edgecolor='yellow', linewidth=2)

    ax.add_collection(pc)
plt.title('Rhea')
plt.show()

In [ ]:
bg = MAPS['RHEA']
fig, ax = bg.figure(figsize=(20, 10))

patches = []

for p in range(len(pixel)):
    hapke.generate_patches(patches,pixel[p])

pc = PatchCollection(patches, alpha=0.4, edgecolor='yellow', linewidth=2)

ax.add_collection(pc)

In [ ]:
plt.show()

In [ ]:
for p in range(len(pixel)):
    print('KEY: ' + str(pixel_key[p]) + '  AREA: ' + str(pixel[p].area))

Plot spectra

In [ ]:
IF_1 = {}
wav1 = {}

fig, ax = plt.subplots()
ax.set_xlabel(r'Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_title('')
ax.grid()


for p in range(len(pixel)):
    ax.plot(pixel[p].wvlns,pixel[p].spectrum)
    #print([pixel[p].phase,pixel[p].eme,pixel[p].inc])


plt.show()

Make fits and store relevant data
ERROR CONSIDERATION: If the error of D or theta is greater than 50% the found value it is not taken into account

In [37]:
IF = {}
wav1 = {}
angles = {}

n_fits = 20

fits = {}
model_IF = {}

D_fit = []
error_D = []
theta_fit = []
error_theta = []

legend_fit = []
legend_cryst = []

area_norm_1 = []
loc2 = []
err_loc2 = []

area = []

for p in range(len(pixel)):

    print(str(p) + '/' + str(len(pixel)))

    area.append(pixel[p].area)

    IF[pixel_key[p]] = pixel[p].spectrum
    wav1[pixel_key[p]] = pixel[p].wvlns

    angles[pixel_key[p]] = [np.deg2rad(pixel[p].eme),np.deg2rad(pixel[p].inc),np.deg2rad(pixel[p].phase)]

    random_fit = hapke.random_fit(n_fits,wav, angles[pixel_key[p]], IF[pixel_key[p]], wav1[pixel_key[p]],n_c,k_c,n_a,k_a)

    print([random_fit['dif'],random_fit['n_sol'],random_fit['Unique']])

    fits[pixel_key[p]] = random_fit['fit']

    model_IF[pixel_key[p]] = hapke.hapke_model_mixed(fits[pixel_key[p]].x,wav,angles[pixel_key[p]],n_c,k_c,n_a,k_a)['IF']

    # CALCULATE ERRORS
    errors = hapke.print_error_correlation(fits[pixel_key[p]])

    if errors[0] < fits[pixel_key[p]].x[0] / 2 and errors[1] < fits[pixel_key[p]].x[1] / 2:
        a = 1
    error_D.append(errors[1])
    error_theta.append(errors[0])


    legend_fit.append(pixel_key[p])
    D_fit.append(fits[pixel_key[p]].x[1])
    theta_fit.append(fits[pixel_key[p]].x[0])

    legend_cryst.append(pixel_key[p])

    area_norm_1.append(hapke.cryst_area_one_peak(IF[pixel_key[p]],wav1[pixel_key[p]])['area'] /hapke.cryst_area_one_peak(model_IF[pixel_key[p]],wav)['area'])

    loc2.append(hapke.cryst_area(IF[pixel_key[p]],wav1[pixel_key[p]])['loc_2'][0])
    err_loc2.append(hapke.cryst_area(IF[pixel_key[p]],wav1[pixel_key[p]])['loc_2'][1])

0/8
[1.7727308929262335, 1.7727308929293513, 1.7727308929969117, 1.772730893014868, 1.7727308930564996, 1.772730893059658, 1.7727308930671772, 1.7727308930752201, 1.772730893105527, 1.7727308931060026, 1.7727308931228063, 1.772730893125139, 1.7727308931578925, 1.7727308931883634, 1.7727308931918442, 1.7727308932380172, 1.772730893392614, 1.7727308938760586, 1.7727308941361337, 1.7727308943194262]
[[2.995829665453087e-06, 1.0890925447587648e-09, 1.3931926723387278e-09], 1, True]
1/8
[1.591357333557192, 1.5913573335956217, 1.5913573336015634, 1.5913573336321825, 1.59135733365288, 1.5913573336598168, 1.5913573336620264, 1.5913573336633122, 1.5913573336997695, 1.5913573337198628, 1.5913573337542883, 1.5913573337877738, 1.5913573338094082, 1.5913573338125013, 1.5913573338174802, 1.591357333860701, 1.5913573338661333, 1.5913573339503015, 1.5913573345960743, 1.5913573348404655]
[[2.6193504232274734e-06, 8.637054138977947e-10, 1.2832734874734797e-09], 1, True]
2/8
[1.654898945364001, 1.6548989

In [38]:
# Shapiro-Wilk test
_, p_value = stats.shapiro(np.array(D_fit))
if p_value > 0.05:
    print("GRAIN SIZE may follow a Gaussian distribution (p-value=%.4f)" % p_value)
else:
    print("GRAIN SIZE does NOT follow a Gaussian distribution (p-value=%.4f)" % p_value)

_, p_value = stats.shapiro(np.array(theta_fit))
if p_value > 0.05:
    print("ROUGHNESS may follow a Gaussian distribution (p-value=%.4f)" % p_value)
else:
    print("ROUGHNESS does NOT follow a Gaussian distribution (p-value=%.4f)" % p_value)

_, p_value = stats.shapiro(np.array(loc2))
if p_value > 0.05:
    print("LOC2 may follow a Gaussian distribution (p-value=%.4f)" % p_value)
else:
    print("LOC2 does NOT follow a Gaussian distribution (p-value=%.4f)" % p_value)

_, p_value = stats.shapiro(np.array(area_norm_1))
if p_value > 0.05:
    print("AREA may follow a Gaussian distribution (p-value=%.4f)" % p_value)
else:
    print("AREA does NOT follow a Gaussian distribution (p-value=%.4f)" % p_value)


GRAIN SIZE does NOT follow a Gaussian distribution (p-value=0.0322)
ROUGHNESS may follow a Gaussian distribution (p-value=0.3523)
LOC2 may follow a Gaussian distribution (p-value=0.6527)
AREA may follow a Gaussian distribution (p-value=0.0893)


In [39]:
print(f'LAT LON = {lat} - {lon}')

print(f'#Pixels w valid D/theta= {len(legend_fit)}')
print(f'#Pixels = {len(legend_cryst)}')

ar = hapke.array_mean(area)
print(f'Pix Area = {ar[0]:.4f} ± {ar[1]:.4f}')

D = hapke.array_mean(D_fit, error_D)
print(f'D = {D[0]*1000000:.4f} ± {D[1]*1000000:.4f}')

theta = hapke.array_mean(theta_fit, error_theta)
print(f'theta = {np.rad2deg(theta[0]):.4f} ± {np.rad2deg(theta[1]):.4f}')

loc_2m = hapke.array_mean(loc2,err_loc2)
print(f'Loc 2m = {loc_2m[0]:.4f} ± {loc_2m[1]:.4f}')

norm_area = hapke.array_mean(area_norm_1)
print(f'Norm area = {norm_area[0]:.4f} ± {norm_area[1]:.4f}')

LAT LON = 90 - 0
#Pixels w valid D/theta= 8
#Pixels = 8
Pix Area = 5612.2097 ± 1728.5719
D = 65.5299 ± 9.0779
theta = 67.4261 ± 2.0134
Loc 2m = 2.0259 ± 0.0004
Norm area = 0.8509 ± 0.0357


In [ ]:
fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1, sharex = True)

for i in range(len(pixel_flyby)):
    if pixel_flyby[i] == 'R1':
        marker = '-o'
        color = colors[0]
    elif pixel_flyby[i] == 'R4':
        marker = '-x'
        color = colors[1]


    ax1.errorbar(range(len(pixel_flyby))[i],np.array(D_fit)[i]*1000000,yerr = np.array(error_D)[i]*1000000, fmt =marker,c=color)

    ax2.errorbar(range(len(pixel_flyby))[i],np.rad2deg(theta_fit)[i],yerr = np.rad2deg(error_theta)[i], fmt =marker,c=color)

    ax3.errorbar(range(len(pixel_flyby))[i],loc2[i],yerr = err_loc2[i], fmt =marker,c=color)

    ax4.errorbar(range(len(pixel_flyby))[i],area_norm_1[i], fmt =marker,c=color)

ax1.set_ylabel(r'$D$ [$\mu$m]')
ax2.set_ylabel(r'$\bar{\theta}$ [deg]')
ax3.set_ylabel(r'2-$\mu$m location')
ax4.set_ylabel(r'Area [-]')

ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()

plt.show()

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1)

ax1.errorbar(legend_fit, np.array(D_fit)*1000000,yerr=np.array(error_D)*1000000, fmt='-o')
ax1.set_ylabel(r'D')

ax2.errorbar(legend_fit, np.array(theta_fit),yerr=np.array(error_theta), fmt='-o')
ax2.set_ylabel(r'$\bar{\theta}$')
ax2.set_xlabel(r'Experiment')
ax1.grid()
ax2.grid()
plt.legend()

plt.show()

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1)

ax1.errorbar(legend_cryst, np.array(cryst_2_loc), fmt='-o')
ax1.set_ylabel(r'Loc 2 micron peak')

ax2.errorbar(legend_cryst, np.array(cryst_area_1), fmt='-o')
ax2.errorbar(legend_cryst, np.array(cryst_area), fmt='-x')
ax2.set_ylabel(r'Area')

ax3.errorbar(legend_cryst, np.array(cocient), fmt='-o')
ax3.set_ylabel(r'Cocient')

ax1.grid()
ax2.grid()
ax3.grid()

plt.show()

In [ ]:
from scipy.stats import linregress

slope, intercept, r_value, p_value, std_err = linregress(cryst_2_loc, cocient)
r2_c = r_value**2
print("R-squared (R²) cociente:", r2_c**2)

slope, intercept, r_value, p_value, std_err = linregress(cryst_2_loc, cryst_area_1)
r2_a1 = r_value**2
print("R-squared (R²) area 1:", r2_a1**2)

slope, intercept, r_value, p_value, std_err = linregress(cryst_2_loc, cryst_area)
r2_a2 = r_value**2
print("R-squared (R²) area 2:", r2_a2**2)

slope, intercept, r_value, p_value, std_err = linregress(cocient, cryst_area_1)
r2_a1_c = r_value**2
print("R-squared (R²) cociente/area:", r2_a1_c**2)

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1)

ax1.errorbar(cryst_2_loc, cocient, fmt='o',label = f'R^2 = {r2_c:.4f}')
ax1.set_ylabel(r'coecient')
ax1.legend()

ax2.errorbar(cryst_2_loc, cryst_area_1, fmt='o',label = f'Area 1 peak. R^2 = {r2_a1:.4f}')
ax2.errorbar(cryst_2_loc, cryst_area, fmt='x',label = f'Area 2 peak. R^2 = {r2_a2:.4f}')
ax2.set_ylabel(r'Area')
ax2.set_xlabel(r'2 micron loc')
ax1.grid()
ax2.grid()
plt.legend()

plt.show()

In [ ]:
# Shapiro-Wilk test
_, p_value = stats.shapiro(np.array(theta_fit))
if p_value > 0.05:
    print("Data may follow a Gaussian distribution (p-value=%.4f)" % p_value)
else:
    print("Data does not follow a Gaussian distribution (p-value=%.4f)" % p_value)

In [ ]:
cube = cube_R1['1511717019_1']

In [ ]:
bg = MAPS[cube.target_name]
fig, ax = bg.figure(figsize=(20, 10))

ax.add_collection(cube.pixels(2.03, vmin=.05, vmax=.18))
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))

plt.imshow(cube@3.1, extent=cube.extent, cmap='gray', vmin=0, vmax=.18)

plt.colorbar(extend='max', label='I/F')

plt.scatter(18, 19, s=150)

plt.xlabel(cube.slabel)
plt.ylabel(cube.llabel)

plt.xticks(cube.sticks)
plt.yticks(cube.lticks)
plt.show()

In [ ]:
pixel1 = cube@(18, 19)
print(pixel1.lonlat)

In [ ]:
pixel1 = cube@(13, 50)

e, i, phase = [np.deg2rad(pixel1.eme),np.deg2rad(pixel1.inc),np.deg2rad(pixel1.phase)]


angles = [e,i,phase]
IF1 = pixel1.spectrum
wav1 = pixel1.wvlns
ini_par = [0.1,0.00001]

body = 0

optimized_parameters = optimize.least_squares(
    hapke.cost_function_mixed, ini_par, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,0,2.8)
)

optimized_values = optimized_parameters.x
hapke.print_error_correlation(optimized_parameters)

print('')

ini_mass = 0.1

cryst_fit = optimize.least_squares(
        hapke.cost_function_mixed_mass_fraction, ini_mass, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,optimized_values,1.45,1.8,1.8), bounds=([0],[1]),
)

hapke.print_error_correlation(cryst_fit)

print('1.2-1.65 RATIO: ' + str(hapke.crystallinity_coecient(IF1,wav1)))

print('')
print('Longitude: ' + str(pixel1.lon))
print('Latitude: ' + str(pixel1.lat))
print('Phase: ' + str(pixel1.phase))

In [ ]:
print(hapke.cryst_area(IF1,wav1)['area'])
print(hapke.cryst_area(hapke.hapke_model_mixed(optimized_values, wav, angles, n_c, k_c, n_a, k_a)['IF'],wav)['area'])
print(hapke.cryst_area(IF1,wav1)['area']/hapke.cryst_area(hapke.hapke_model_mixed(optimized_values, wav, angles, n_c, k_c, n_a, k_a)['IF'],wav)['area'])

In [ ]:
fig, ax = plt.subplots()
ax.plot(wav1, IF1)
ax.plot(wav, hapke.hapke_model_mixed(optimized_values, wav, angles, n_c, k_c, n_a, k_a)['IF'], ls = ':')
ax.plot(wav, hapke.hapke_model_mixed_mass_fraction(cryst_fit.x, wav, angles, n_c, k_c, n_a, k_a,optimized_values)['IF'], ls = '-.')
ax.set_xlabel(r'Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_title('')
ax.set_xlim(1.1,3.3)
ax.grid()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#ax.legend()
plt.show()

In [ ]:
pixel1 = cube@(30,15)

e, i, phase = [np.deg2rad(pixel1.eme),np.deg2rad(pixel1.inc),np.deg2rad(pixel1.phase)]


angles = [e,i,phase]
IF1 = pixel1.spectrum
wav1 = pixel1.wvlns
ini_par = [0.1,np.deg2rad(20),0.1,0.00005,0.5]

body = cube.target_name

optimized_parameters = optimize.least_squares(
    hapke.cost_function_mixed_no_weight, ini_par, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,1.5,1.85,body), bounds=([0.001,0,0,0,0], [1,np.deg2rad(44.9),0.74,0.001,1], ), x_scale='jac',
)

optimized_values = optimized_parameters.x
hapke.print_error_correlation(optimized_parameters)

print('')
print('Longitude: ' + str(pixel1.lon))
print('Latitude: ' + str(pixel1.lat))
print('Phase: ' + str(pixel1.phase))

This cell below created 20 randomly generated initial parameters and store the results of the fits on a dictionary fit_dict. The variable count adds one every time a set of generated initial parameters leads to the same solution than the previous one.

In [ ]:
optimized_values = optimized_parameters.x

old_hapke = hapke.hapke_model_mixed(ini_par,wav, angles, n_c,k_c,n_a,k_a,body)['IF']

new_hapke = hapke.hapke_model_mixed(optimized_values,wav, angles, n_c,k_c,n_a,k_a,body)['IF']

fig, ax = plt.subplots(figsize = (8,3))
ax.plot(wav, old_hapke, label = 'Hapke Initial')
ax.plot(wav, new_hapke, label = 'Hapke Optimized')
ax.plot(wav1, IF1, label = 'Pixel')
ax.set_xlabel('Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_xlim(1.1,3.2)
ax.set_title('')
ax.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

plt.imshow(cube@2.03, extent=cube.extent, cmap='gray', vmin=0, vmax=.18)

plt.colorbar(extend='max', label='I/F')

plt.scatter(13, 50, s=150)

plt.xlabel(cube.slabel)
plt.ylabel(cube.llabel)

plt.xticks(cube.sticks)
plt.yticks(cube.lticks)
plt.show()

These coming two cells plot the surface response: the residuals compared to the spectrum taken by VIMS. Note that wa1 and IF1 need to be defined beforehand and that the function hapke_mixed needs to be changes to be 2D in parameters

In [ ]:
# SURFACE RESPONSE OF ENCELADUS SPECTRA

real_par = [0.51,0.00007]

m_range = np.linspace(0.001,1,30)
phi_range = np.linspace(0.001,0.74,30)

residuals = np.zeros((len(phi_range),len(m_range)))


interp_func_2 = interp1d(wav1, IF1, bounds_error=False)

interpolated_lab = interp_func_2(wav)

for i in range(len(phi_range)):
    for j in range(len(m_range)):
        param = [phi_range[i], m_range[j]]
        dif = (interpolated_lab - hapke.hapke_model_mixed(param,wav,angles,n_c,k_c,n_a,k_a,body)['IF'])
        dif = dif[~np.isnan(dif)]
        residuals[i,j] = np.linalg.norm(dif)

In [ ]:
# 3D PLOT OF THE SURFACE RESPONSE

phi_range, m_range = np.meshgrid(phi_range, m_range)
# Define the data

# Create a 3D plot
fig = plt.figure(figsize = (4,4))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(phi_range, m_range, residuals.T, cmap='viridis')
ax.scatter(0.16933,0.502, c = 'green', label = 'Real solution' )
ax.legend()
ax.set_xlabel(r'$\phi$ [-]')
ax.set_ylabel(r'$m$ [-]')
ax.set_zlabel('Residuals')
# Show the plot
plt.show()